<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/02_nfl_predict_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
from sklearn.decomposition import PCA

In [48]:
pd.set_option('display.max_columns', None)

# 02. Feature Engineering

## Reading data

In [49]:
PATH = 'https://raw.githubusercontent.com/rafabandoni/nfl-predict/refs/heads/main/data/output/'

In [50]:
treated_df = pd.read_parquet(PATH + 'treated_df.parquet')
treated_df.head()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_ho

In [51]:
featured_df = treated_df.copy()

## Lag

* Lag is a feature with time information

### Last score lag playing home and away

In [52]:
def get_lag_by_team_date(data, lag, home=True):
  final_lagged_df = pd.DataFrame()
  is_home_team = 'team_home' if home==True else 'team_away'
  is_home_score = 'score_home' if home==True else 'score_away'
  new_column_name = 'home_lag_' if home==True else 'away_lag_'

  for team in data[is_home_team].unique():
    lag_df = data[data[is_home_team] == team]
    lag_df = lag_df[['schedule_date', is_home_team, is_home_score]]
    lag_df = lag_df.sort_values([is_home_team, 'schedule_date'])
    lag_df = lag_df.set_index([is_home_team, 'schedule_date']).shift(lag).reset_index()
    final_lagged_df = pd.concat([final_lagged_df, lag_df], axis=0, ignore_index=True)
  final_lagged_df.rename(columns={
      is_home_score : f'{new_column_name}{lag}'
  }, inplace=True)
  return final_lagged_df

In [53]:
df_home_lag_1 = get_lag_by_team_date(treated_df, 1)
df_home_lag_2 = get_lag_by_team_date(treated_df, 2)
df_home_lag_3 = get_lag_by_team_date(treated_df, 3)

df_home_lag = df_home_lag_1.merge(df_home_lag_2,
                                  on=['team_home', 'schedule_date'],
                                  how='left')
df_home_lag = df_home_lag.merge(df_home_lag_3,
                                  on=['team_home', 'schedule_date'],
                                  how='left')

In [54]:
df_away_lag_1 = get_lag_by_team_date(treated_df, 1, home=False)
df_away_lag_2 = get_lag_by_team_date(treated_df, 2, home=False)
df_away_lag_3 = get_lag_by_team_date(treated_df, 3, home=False)

df_away_lag = df_away_lag_1.merge(df_away_lag_2,
                                  on=['team_away', 'schedule_date'],
                                  how='left')
df_away_lag = df_away_lag.merge(df_away_lag_3,
                                  on=['team_away', 'schedule_date'],
                                  how='left')

In [55]:
featured_df = featured_df.merge(df_home_lag,
                                on=['team_home', 'schedule_date'],
                                how='left')
featured_df = featured_df.merge(df_away_lag,
                                on=['team_away', 'schedule_date'],
                                how='left')
featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_ho

### Wins and loses of those teams historically

In [56]:
historic = {}

def calculate_historic(row):
    team_1, team_2 = sorted([row['team_home'], row['team_away']])
    key = (team_1, team_2)
    if key not in historic:
        historic[key] = {'victories_team_1': 0, 'victories_team_2': 0, 'matches': 0}
    historic[key]['matches'] += 1

    if row['home_winner']:
      if row['team_home'] == team_1:
        historic[key]['victories_team_1'] += 1
      else:
        historic[key]['victories_team_2'] += 1
    else:
      if row['team_home'] == team_1:
        historic[key]['victories_team_2'] += 1
      else:
        historic[key]['victories_team_1'] += 1

    if row['team_home'] == team_1:
       return historic[key]['victories_team_1'] / historic[key]['matches']
    else:
      return historic[key]['victories_team_2'] / historic[key]['matches']

featured_df['home_win_historic_confront'] = featured_df.apply(calculate_historic, axis=1)
featured_df['away_win_historic_confront'] = 1 - featured_df['home_win_historic_confront']

## Average result feature

Average points last 3 games

In [57]:
def last_three_games_result(featured_df, home=True):
    team = 'team_home' if home==True else 'team_away'
    score = 'score_home' if home==True else 'score_away'
    df = featured_df[['schedule_date', team, score]].sort_values([team, 'schedule_date'])
    df[f'last_3_scores_{team}'] = df[score].rolling(window=3).mean().shift(1)
    return df

home_team_avg = last_three_games_result(featured_df)
away_team_avg = last_three_games_result(featured_df, home=False)

featured_df = featured_df.merge(home_team_avg[['schedule_date', 'team_home', 'last_3_scores_team_home']],
                                on=['schedule_date', 'team_home'],
                                how='left')
featured_df = featured_df.merge(away_team_avg[['schedule_date', 'team_away', 'last_3_scores_team_away']],
                                on=['schedule_date', 'team_away'],
                                how='left')
featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_ho

Average points per game last season

In [58]:
slice_df = featured_df.copy()
slice_df['schedule_season'] = slice_df['schedule_season'] + 1

home_score_per_season = slice_df[['schedule_season',
                                     'team_home',
                                     'score_home']].groupby(['schedule_season',
                                                             'team_home'], as_index=False).mean()
away_score_per_season = slice_df[['schedule_season',
                                     'team_away',
                                     'score_away']].groupby(['schedule_season',
                                                             'team_away'], as_index=False).mean()

home_score_per_season.rename(columns={'score_home' : 'avg_score_home_last_season'}, inplace=True)
away_score_per_season.rename(columns={'score_away' : 'avg_score_away_last_season'}, inplace=True)

featured_df = featured_df.merge(home_score_per_season,
                                on=['schedule_season', 'team_home'],
                                how='left')
featured_df = featured_df.merge(away_score_per_season,
                                on=['schedule_season', 'team_away'],
                                how='left')

featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_ho

Total points last season

In [59]:
slice_df = featured_df.copy()
slice_df['schedule_season'] = slice_df['schedule_season'] + 1

home_score_per_season = slice_df[['schedule_season',
                                     'team_home',
                                     'score_home']].groupby(['schedule_season',
                                                             'team_home'], as_index=False).sum()
away_score_per_season = slice_df[['schedule_season',
                                     'team_away',
                                     'score_away']].groupby(['schedule_season',
                                                             'team_away'], as_index=False).sum()

home_score_per_season.rename(columns={'score_home' : 'total_score_home_last_season'}, inplace=True)
away_score_per_season.rename(columns={'score_away' : 'total_score_away_last_season'}, inplace=True)

featured_df = featured_df.merge(home_score_per_season,
                                on=['schedule_season', 'team_home'],
                                how='left')
featured_df = featured_df.merge(away_score_per_season,
                                on=['schedule_season', 'team_away'],
                                how='left')

featured_df.tail()

,schedule_date,schedule_season,schedule_playoff,team_home,score_home,score_away,team_away,stadium_neutral,home_winner,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_40+_home_,special_teams_kickoff_returns_lng_ho

Average in-game stats

In [61]:
featured_df.columns[9:]

Index(['defense_downs_3rd_att_home_', 'defense_downs_3rd_md_home_',
       'defense_downs_4th_att_home_', 'defense_downs_4th_md_home_',
       'defense_downs_rec_1st_home_', 'defense_downs_rec_1st_perc_home_',
       'defense_downs_rush_1st_home_', 'defense_downs_rush_1st_perc_home_',
       'defense_downs_scrm_plys_home_', 'year_home_',
       ...
       'away_lag_2', 'away_lag_3', 'home_win_historic_confront',
       'away_win_historic_confront', 'last_3_scores_team_home',
       'last_3_scores_team_away', 'avg_score_home_last_season',
       'avg_score_away_last_season', 'total_score_home_last_season',
       'total_score_away_last_season'],
      dtype='object', length=347)

## Data Leakage avoidance

We might be aware of **data leakage**.

Examples of data leakage in machine learning models:
* Target leakage: When a model learns from data that contains information about the target variable
* Data preprocessing leakage: When information from the test set is used during data preprocessing
* Using the same dataset for training and testing: This is an example of overfitting, which can lead to overoptimistic performance estimates
* Using future data to make predictions about the past: This can happen in time series problems

### Removing score information

# Preprocessing

## Principal Component Analysis(PCA)

PCA is an unsupervised learning algorithm, meaning it doesn’t require prior knowledge of target variables. It’s commonly used in exploratory data analysis and machine learning to simplify datasets without losing critical information.

Know more: https://www.geeksforgeeks.org/principal-component-analysis-pca/

X, y and train test split before making preprocessing to avoid data leakage

In [16]:
X = featured_df.select_dtypes(include='number')
X.dropna(inplace=True)

In [17]:
# Mean
X_mean = X.mean()

# Standard deviation
X_std = X.std()

# Standardization
Z = (X - X_mean) / X_std

In [28]:

pca = PCA(n_components=0.7)
pca.fit(Z)
x_pca = pca.transform(Z)

# Create the dataframe
df_pca1 = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(len(pca.components_))])
df_pca1

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42
0,-3.332779,3.580450,-5.547129,0.760488,-1.541298,-0.554102,5.858198,-8.071817,0.149099,-1.728525,2.797968,0.736869,-3.540646,-4.713654,0.704986,3.379522,-0.638328,-0.283404,-0.566847,2.759973,-0.219268,0.219404,1.265874,2.831195,2.590748,-1.109382,-1.115833,0.382245,0.529850,-0.407524,1.762129,1.354695,0.295430,1.605046,0.393941,-0.069772,-1.560603,0.314356,-0.719456,-0.145616,-0.055639,1.267959
1,-12.011516,0.462317,-1.169192,3.308653,3.019664,2.789588,-0.682031,1.012113,3.287993,-2.281227,1.949365,-2.734720,0.669693,-2.518663,1.318554,0.057693,-3.425658,2.021375,-0.337790,2.643036,1.315725,-1.716056,0.985843,-3.626068,1.064711,-1.237952,-3.987945,0.897647,-0.515604,-0.408054,1.098785,-2.625720,-1.595239,1.319922,-0.604015,1.803953,1.007748,-0.199077,-1.828795,-2.431123,0.189698,0.151374
2,-4.546710,1.609716,-3.682022,-3.325684,1.251979,-0.880518,-0.849470,-4.885924,3.234969,-0.353215,0.039609,-0.044995,2.124431,-7.634449,0.119363,-0.418270,0.553629,-1.924578,-0.252243,0.454265,-0.743903,0.494647,3.221296,1.150250,-2.726321,1.399007,0.981732,-0.663157,-0.304376,-0.968446,-1.335021,-0.349838,-0.430482,-0.414791,-1.932271,-0.430221,1.916703,-0.470134,1.259059,-1.693970,-0.777360,1.791502
3,-8.535605,7.119972,-6.431080,-1.982848,-1.185368,0.200927,3.169475,-3.765244,-0.278390,-0.496245,-3.185054,-1.865281,-2.060270,-2.131130,2.058761,-3.844649,0.666651,-0.734670,2.167573,3.505303,-0.085455,0.152943,1.564344,2.219128,-0.946533,-0.421784,-0.499810,0.803125,0.672148,0.520591,-0.783293,-1.113513,1.472991,-0.117090,1.534602,-0.673563,-2.050148,1.141078,-0.886854,1.498494,2.053818,0.381016
4,-10.186177,0.993890,-5.116444,-2.754739,-3.524706,5.636805,-1.465188,-4.655712,2.692232,-3.741901,0.467279,0.098373,1.321980,3.425269,1.484332,1.244492,-0.108411,-2.066484,0.749411,0.972007,-5.160052,-1.363464,0.958736,3.525893,-1.587266,1.779650,1.032701,-0.416680,1.456808,1.608841,-1.378363,-1.952699,-1.265337,-2.109108,0.373604,-2.531787,1.359689,-0.369549,-1.296767,0.861436,0.820551,0.900079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,6.434914,0.342244,1.428066,-7.581533,-10.282632,0.627770,-2.383158,-0.526937,3.101059,-1.493109,-1.843739,-1.101596,-0.434766,0.384638,-0.575688,-1.848980,-0.821582,1.564743,1.657142,-1.664209,1.402390,2.644729,-3.393473,-0.424286,-2.156259,1.714103,-1.282011,-1.666202,-1.742746,0.645815,0.251278,1.088257,-0.474192,2.894463,1.301774,-0.887282,-0.398734,-1.499937,-2.670218,-0.423933,0.621998,-0.686023
3704,9.132686,0.458870,7.334357,-1.480473,-4.448425,0.094593,3.065766,3.515908,2.674255,0.901148,-2.275111,3.418100,2.208471,3.581858,3.172931,-2.725617,1.264721,0.298947,1.326343,-2.685223,0.785514,-0.114252,-2.216062,-3.454440,2.457647,2.054964,-0.708673,0.705612,1.173721,-1.154628,-1.731792,2.132819,-0.677686,0.448290,1.904048,0.554560,2.789234,1.704424,-2.492998,1.951919,1.014169,1.246917
3705,6.986268,-3.590161,10.230472,5.965377,-1.438028,0.693167,1.885778,3.850692,4.037750,0.477924,-0.765452,1.402791,3.441915,2.273702,2.161956,-2.174526,1.347138,1.943759,2.095653,-0.128648,-1.770320,4.130593,-2.605922,-0.114862,0.581177,-3.143930,-0.709669,0.444224,-0.971664,-0.085166,-0.910936,0.490458,-1.042751,0.790171,0.622359,0.682070,0.053999,-1.217041,-3.011885,4.004862,1.946658,1.302500
3706,5.917467,1.585285,2.623806,-5.063058,-7.187636,-5.198020,-2.852895,-3.386310,1.506676,1.358950,-0.120986,-0.104929,0.033401,-1.294011,0.556274,-0.255936,2.021482,2.807443,-0.056800,-3.771645,-3.667160,2.751327,-2.619535,-0.098960,0.189817,0.530865,0.954809,-1.561176,1.630494,0.564425,1.149887,-0.338890,-2.628207,1.991931,-1.434794,-1.098846,-1.438527,1.641778,0.536723,3.141850,-2.192458,1.351350
